In [1]:
!ls Gas

coteq_gas_2013.csv	  enexis_gas_01012014.csv   rendo_gas_2017.csv
coteq_gas_2014.csv	  enexis_gas_01012015.csv   rendo_gas_2018.csv
coteq_gas_2015.csv	  enexis_gas_01012016.csv   rendo_gas_2019.csv
coteq_gas_2016.csv	  enexis_gas_01012017.csv   stedin_gas_2009.csv
coteq_gas_2017.csv	  enexis_gas_01012018.csv   stedin_gas_2010.csv
coteq_gas_2018.csv	  enexis_gas_01012019.csv   stedin_gas_2011.csv
coteq_gas_2019.csv	  enexis_gas_01012020.csv   stedin_gas_2012.csv
coteq_gas_2020.csv	  liander_gas_01012009.csv  stedin_gas_2013.csv
endinet_gas_01012011.csv  liander_gas_01012010.csv  stedin_gas_2014.csv
endinet_gas_01012012.csv  liander_gas_01012011.csv  stedin_gas_2015.csv
endinet_gas_01012013.csv  liander_gas_01012012.csv  stedin_gas_2016.csv
endinet_gas_01012014.csv  liander_gas_01012013.csv  stedin_gas_2017.csv
endinet_gas_01012015.csv  liander_gas_01012014.csv  stedin_gas_2018.csv
endinet_gas_01012016.csv  liander_gas_01012015.csv  stedin_gas_2019.csv
enduris_gas_01012013.csv  liander_g

In [38]:
!pip install mysql.connector

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.9/11.9 MB 34.2 MB/s eta 0:00:00m eta 0:00:0136m0:00:01
  Preparing metadata (setup.py) ... one
  DEPRECATION: Building 'mysql.connector' using the legacy setup.py bdist_wheel mechanism, which will be removed in a future version. pip 25.3 will enforce this behaviour change. A possible replacement is to use the standardized build interface by setting the `--use-pep517` option, (possibly combined with `--no-build-isolation`), or adding a `pyproject.toml` file to the source tree of 'mysql.connector'. Discussion can be found at https://github.com/pypa/pip/issues/6334
  Created wheel for mysql.connector: filename=mysql_connector-2.2.9-cp313-cp313-linux_x86_64.whl size=247949 sha256=bfb9a771b88d6b33b4cdc6aef923308918449cf52cd189627ae7f5b66c5bdc13
  Stored in directory: /home/caduct/.cache/pip/wheels/4e/c1/23/9423c444a75266a05275c80ebf5c6efe2368388e976b690a0f
Successfully built mysql.connector


In [88]:
!pip install progress

  Using cached progress-1.6.1-py3-none-any.whl.metadata (4.3 kB)
Using cached progress-1.6.1-py3-none-any.whl (9.8 kB)


In [107]:
import pandas as pd
import os
import mysql.connector
from mysql.connector import Error
from progress.bar import IncrementalBar

In [3]:
for filename in os.listdir('./Gas'):
    namelist = filename.split('_')
    if namelist[1] != 'gas':
        namelist[0] = namelist[0][:-3]
        namelist.append(namelist[1])
        namelist[1] = 'gas'
        new_name = '_'.join(namelist)
        os.rename(f"./Gas/{filename}", f"./Gas/{new_name}")

In [12]:
df_list=[]
for filename in os.listdir('./Gas'):
    df = pd.read_csv(f"./Gas/{filename}")
    df['year'] = int(filename[-8:-4])
    df_list.append(df)
    if 'net_manager' not in list(df.columns.values):
        df['net_manager'] = filename.split('_')[0]
        print(df['net_manager'])
df = pd.concat(df_list)


0       enduris
1       enduris
2       enduris
3       enduris
4       enduris
         ...   
9035    enduris
9036    enduris
9037    enduris
9038    enduris
9039    enduris
Name: net_manager, Length: 9040, dtype: object
0       enduris
1       enduris
2       enduris
3       enduris
4       enduris
         ...   
9055    enduris
9056    enduris
9057    enduris
9058    enduris
9059    enduris
Name: net_manager, Length: 9060, dtype: object
0       enduris
1       enduris
2       enduris
3       enduris
4       enduris
         ...   
9004    enduris
9005    enduris
9006    enduris
9007    enduris
9008    enduris
Name: net_manager, Length: 9009, dtype: object


In [13]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3704367 entries, 0 to 5016
Data columns (total 18 columns):
 #   Column                        Dtype  
---  ------                        -----  
 0   net_manager                   object 
 1   purchase_area                 object 
 2   street                        object 
 3   zipcode_from                  object 
 4   zipcode_to                    object 
 5   city                          object 
 6   num_connections               float64
 7   delivery_perc                 float64
 8   perc_of_active_connections    float64
 9   type_conn_perc                object 
 10  type_of_connection            object 
 11  annual_consume                float64
 12  annual_consume_lowtarif_perc  float64
 13  smartmeter_perc               float64
 14  year                          int64  
 15  STANDAARDDEVIATIE             float64
 16  ï»¿NETBEHEERDER               float64
 17  %Defintieve aansl (NRM)       float64
dtypes: float64(9), int64(1), objec

In [14]:
df = df.drop(['STANDAARDDEVIATIE', 'ï»¿NETBEHEERDER', '%Defintieve aansl (NRM)', 'smartmeter_perc'], axis=1)

In [15]:
df.isna().sum().sort_values()

net_manager                          0
street                               0
city                                 0
zipcode_to                           0
year                                 0
annual_consume                       0
zipcode_from                         1
num_connections                     10
perc_of_active_connections        5017
delivery_perc                     5017
annual_consume_lowtarif_perc     44787
type_of_connection               82976
type_conn_perc                  144258
purchase_area                   235936
dtype: int64

In [33]:
names = df['net_manager'].value_counts().reset_index()['net_manager'].tolist()
id_dict={names[i]:i for i in range(len(names))}
id_dict

{'GAS Liander': 0,
 'Enexis B.V.': 1,
 8716892000005: 2,
 'Liander NB': 3,
 8716874000009: 4,
 'Enexis': 5,
 8716886000004: 6,
 'Endinet Oost Brabant NV': 7,
 'Endinet regio Eindhoven BV': 8,
 8716921000006: 9,
 8716912000008: 10,
 'Coteq Netbeheer BV': 11,
 8716892750009: 12,
 8716892700004: 13,
 8717127750009: 14,
 8716892740000: 15,
 8716892710003: 16,
 'enduris': 17,
 'westland-infra': 18,
 8716892720002: 19,
 8716925000002: 20,
 'Cogas Infra & Beheer BV': 21,
 8716924000003: 22}

In [29]:
df['net_manager_id'] = df['net_manager'].replace(id_dict)
df.head(5)

/tmp/ipykernel_6348/104205115.py:1: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['net_manager_id'] = df['net_manager'].replace(id_dict)


,net_manager,purchase_area,street,zipcode_from,zipcode_to,city,num_connections,delivery_perc,perc_of_active_connections,type_conn_perc,type_of_connection,annual_consume,annual_consume_lowtarif_perc,year,net_manager_id
0,Endinet regio Eindhoven BV,871718518003011971,de Wieken,0000AA,5091AB,VELDHOVEN,36.0,100.0,100.0,92,G4,1760.0,0.0,2015,8
1,Endinet regio Eindhoven BV,871718518003011971,Margrietstraat,5091AC,5091AC,OOST WEST EN MIDDELBEERS,21.0,100.0,100.0,67,G4,1626.0,0.0,2015,8
2,Endinet regio Eindhoven BV,871718518003011971,Margrietstraat,5091AD,5091AD,OOST WEST EN MIDDELBEERS,14.0,100.0,100.0,100,G4,1178.0,0.0,2015,8
3,Endinet regio Eindhoven BV,871718518003011971,Margrietstraat,5091AE,5091AE,OOST WEST EN MIDDELBEERS,19.0,100.0,100.0,74,G4,1727.0,0.0,2015,8
4,Endinet regio Eindhoven BV,871718518003011971,Rozenplein,5091AG,5091AG,OOST WEST EN MIDDELBEERS,10.0,100.0,100.0,70,G6,2251.0,0.0,2015,8
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5012,8716912000008,871718518003012077,Wateren,8438SB,8438SB,WATEREN,25.0,NaN,NaN,72,G4,3218.0,NaN,2019,10
5013,8716912000008,871718518003012077,Schurerslaan,8438SC,8438SC,WATEREN,43.0,NaN,NaN,88,G4,1546.0,NaN,2019,10
5014,8716912000008,871718518003012077,Lange Drift,8438SE,8438SG,WATEREN,18.0,NaN,NaN,72,G4,2012.0,NaN,2019,10
5015,8716912000008,871718518003012077,Huenderweg,8438SH,8438ST,WATEREN,85.0,NaN,NaN,98,G4,1928.0,NaN,2019,10


In [119]:
def clean_nan_values(value):
    """Преобразует NaN/NaT в None"""
    if pd.isna(value):
        return None
    return value

def create_database(df, manager_dict):
    try:
        connection = mysql.connector.connect(
            host='localhost',
            user='caduct',
            password='Apple2013',
            auth_plugin='mysql_native_password',
            database='gas_db')
        
        if connection.is_connected():
            cursor = connection.cursor()
            
            cursor.execute("""
            CREATE TABLE IF NOT EXISTS managers_mapping (
                manager_id INT PRIMARY KEY,
                manager_name VARCHAR(255) NOT NULL UNIQUE
            )
            """)
            
            for manager_name, manager_id in manager_dict.items():
                cursor.execute("""
                INSERT IGNORE INTO managers_mapping (manager_id, manager_name)
                VALUES (%s, %s)
                """, (manager_id, manager_name))
            
            for manager_name, manager_id in manager_dict.items():
                table_name = f"net_manager_{manager_id}"
                
                create_table_sql = f"""
                CREATE TABLE IF NOT EXISTS {table_name} (
                    id INT AUTO_INCREMENT PRIMARY KEY,
                    manager_id INT NOT NULL,
                    purchase_area VARCHAR(255),
                    street VARCHAR(255),
                    zipcode_from VARCHAR(20),
                    zipcode_to VARCHAR(20),
                    city VARCHAR(255),
                    num_connections FLOAT,
                    delivery_perc FLOAT,
                    perc_of_active_connections FLOAT,
                    type_conn_perc VARCHAR(50),
                    type_of_connection VARCHAR(100),
                    annual_consume FLOAT,
                    annual_consume_lowtarif_perc FLOAT,
                    year INT,

                    FOREIGN KEY(manager_id)
                    REFERENCES managers_mapping(manager_id)
                );
                """
                cursor.execute(create_table_sql)
            
            connection.commit()
            print("БД успешно создана!")
            
            if df is not None:
                total_rows = len(df)
                print(f"Всего записей для обработки: {total_rows}")
                
                bar = IncrementalBar('Перенос данных', max=total_rows, suffix='%(percent)d%% [%(elapsed_td)s / %(eta_td)s]')
                
                records_by_manager = {manager_id: 0 for manager_id in manager_dict.values()}
                
                for index, row in df.iterrows():
                    manager_name = clean_nan_values(row.get('net_manager'))
                    
                    if manager_name is None:
                        bar.next()
                        processed += 1
                        continue
                    
                    manager_id = manager_dict.get(manager_name)
                    
                    if manager_id is None:
                        bar.next()
                        processed += 1
                        continue
                    
                    table_name = f"net_manager_{manager_id}"
                    
                    insert_data = (
                        manager_id,  
                        clean_nan_values(row.get('purchase_area')),
                        clean_nan_values(row.get('street')),
                        clean_nan_values(row.get('zipcode_from')),
                        clean_nan_values(row.get('zipcode_to')),
                        clean_nan_values(row.get('city')),
                        clean_nan_values(row.get('num_connections')),
                        clean_nan_values(row.get('delivery_perc')),
                        clean_nan_values(row.get('perc_of_active_connections')),
                        clean_nan_values(row.get('type_conn_perc')),
                        clean_nan_values(row.get('type_of_connection')),
                        clean_nan_values(row.get('annual_consume')),
                        clean_nan_values(row.get('annual_consume_lowtarif_perc')),
                        clean_nan_values(row.get('year'))
                    )
                    
                    insert_sql = f"""
                    INSERT INTO {table_name} (
                        manager_id, purchase_area, street, zipcode_from, zipcode_to, city,
                        num_connections, delivery_perc, perc_of_active_connections,
                        type_conn_perc, type_of_connection, annual_consume,
                        annual_consume_lowtarif_perc, year
                    ) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
                    """
                    
                    try:
                        cursor.execute(insert_sql, insert_data)
                        records_by_manager[manager_id] += 1
                    except Error as e:
                        print(f"\nОшибка при вставке записи {index}: {e}")
                    
                    bar.next()

                    
 
                connection.commit()

    
    except Error as e:
        print(f"\nКритическая ошибка: {e}")
        if 'connection' in locals():
            connection.rollback()
    finally:
        if 'connection' in locals() and connection.is_connected():
            cursor.close()
            connection.close()

In [120]:
create_database(df, id_dict)

БД успешно создана!
Всего записей для обработки: 3704367


In [121]:
df.shape

(3704367, 15)